In [ ]:
%pip install openai
%pip install videodb

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["ELEVEN_LABS_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

ElevenLabs has a large variety of voices to choose from ([here](https://elevenlabs.io/voice-library)), Once finalized, copy the Voice ID from ElevenLabs

In [ ]:
voiceover_artist_id = "VOICEOVER_ARTIST_ID"

In [ ]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()

In [ ]:
# Upload a video by URL (replace the url with your video)
video = conn.upload(url='https://youtu.be/RcRjY5kzia8')

In [ ]:
# This will provide context for generating the script prompt
video.index_scenes()

Let's view the description of first scene of the video

In [ ]:
scenes = video.get_scenes()
print(f"{scenes[0]['start']} - {scenes[0]['end']}")
print(scenes[0]["response"])

Combine scene descriptions with the script prompt, instructing LLM to create a voiceover script

In [ ]:
import openai

client = openai.OpenAI()

script_prompt = "Here's the data from a scene index for a video about the <very short description of you video>. Study this and then generate a synced script based on the description below."

full_prompt = script_prompt + "\n\n"
for scene in scenes:
  full_prompt += f"- {scene}\n"

openai_res = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": full_prompt}],
)
voiceover_script = openai_res.choices[0].message.content

# If you have ElevenLab's paid plan remove this
voiceover_script = voiceover_script[:2500]

Utilize the generated script to synthesize AI-generated voiceover narration using ElevenLabs API

In [ ]:
import requests


# Call ElevenLabs API to generate voiceover
url = f"https://api.elevenlabs.io/v1/text-to-speech/{voiceover_artist_id}"
headers = {
    "xi-api-key": os.environ.get("ELEVEN_LABS_API_KEY"),
    "Content-Type": "application/json"
}
payload = {
    "model_id": "eleven_monolingual_v1",
    "text": voiceover_script,
    "voice_settings": {
        "stability": 0.5,
        "similarity_boost": 0.5
    }
}
elevenlabs_res = requests.request("POST", url, json=payload, headers=headers)

# Save the audio file
audio_file = "audio.mp3"
CHUNK_SIZE = 1024
with open(audio_file, 'wb') as f:
    for chunk in elevenlabs_res.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)

To use the voiceover generated above, uploading the voiceover file to VideoDB

In [ ]:
audio = conn.upload(file_path=audio_file)

Finally, add the AI-generated voiceover to the original footage using VideoDB's timeline feature

In [ ]:
from videodb.timeline import Timeline
from videodb.asset import VideoAsset, AudioAsset

# Create a timeline object
timeline = Timeline(conn)

# Add the video asset to the timeline for playback
video_asset = VideoAsset(asset_id=video.id)
timeline.add_inline(asset=video_asset)

# Add the audio asset to the timeline for playback
audio_asset = AudioAsset(asset_id=audio.id)
timeline.add_overlay(start=0, asset=audio_asset)

generate a stream of the video


In [ ]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)